# Product Price Comparison
Dataset: https://www.kaggle.com/datasets/prashantk93/supply-chain-management-for-car

In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Load and Transform data

In [271]:
# Load data and select the required feature
usecols = ['SupplierName', 'CarMaker', 'CarModel', 
           'CarColor', 'CarModelYear', 'CarPrice']
df = pd.read_csv('src/Car_SupplyChainManagementDataSet.csv', usecols=usecols)
df.head()

,SupplierName,CarMaker,CarModel,CarColor,CarModelYear,CarPrice
0,Bubbletube,Dodge,Ram 2500,Goldenrod,2007,521963.45
1,Tagopia,Toyota,Tundra,Crimson,2010,672222.04
2,Zoomdog,GMC,Savana 1500,Crimson,2011,504465.72
3,Oozz,Volkswagen,Cabriolet,Fuscia,1990,646077.11
4,Kare,Mercury,Mariner,Teal,2009,699890.24


In [272]:
# Filter CarModel has at least 10 SKU
df = df[df['CarMaker'].map(df['CarMaker'].value_counts()) > 10]

In [273]:
# Create SKU
df['SKU'] = df[df.columns[1:5]].apply(
    lambda x: '-'.join(x.astype(str)), axis=1).apply(
    lambda x: x.upper().replace(' ', ''))
df.drop(['CarMaker', 'CarModel', 'CarColor', 'CarModelYear'], 
        axis=1, inplace=True)
df = df.reindex(columns=['SKU', 'SupplierName', 'CarPrice'])
df.head()

,SKU,SupplierName,CarPrice
0,DODGE-RAM2500-GOLDENROD-2007,Bubbletube,521963.45
1,TOYOTA-TUNDRA-CRIMSON-2010,Tagopia,672222.04
2,GMC-SAVANA1500-CRIMSON-2011,Zoomdog,504465.72
3,VOLKSWAGEN-CABRIOLET-FUSCIA-1990,Oozz,646077.11
4,MERCURY-MARINER-TEAL-2009,Kare,699890.24


### Sample data into HTML form

In [274]:
n_web = 3
n_sample = 800

for i in range(n_web):
    # sample product for sale on the website
    web = df.sample(n=n_sample, random_state=42+i, ignore_index=True)
    # add noise to CarPrice by random with spread about 5% of original price
    web['CarPrice'] = web['CarPrice'].apply(
        lambda x: int(x * (1 + np.random.normal(loc=0, scale=0.05))))
    
    # write html to file
    with open("src/website{0}.html".format(i+1), "w") as f:
        f.write(web.to_html(index=False))

### Web Scraping Products from websites

In [185]:
from bs4 import BeautifulSoup

In [261]:
with open('src/website1.html', 'r') as f:
    html_data = f.read()
soup = BeautifulSoup(html_data)

In [262]:
html_table = soup.find_all('table')

In [263]:
# Extract column names
def extract_column_from_header(row):
    if row.br: row.br.extract()
    if row.a: row.a.extract()
    if row.sup: row.sup.extract()

    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name  

column_names = []
for row in html_table[0].find_all('th'):
    name = extract_column_from_header(row)
    if name is not None and len(name) > 0:
        column_names.append(name)
column_names

['SKU', 'SupplierName', 'CarPrice']

In [151]:
product_dict = dict.fromkeys(column_names)
# Set dict value to empty list
for key in product_dict.keys():
    product_dict[key] = []  

for rows in html_table[0].find_all('tr'):
    # check table heading
    if rows.th:
        continue
    row = rows.find_all('td')
    for i, col_name in enumerate(column_names):
        product_dict[col_name].append(row[i].contents[0])

### Merge data from each websites

In [275]:
website_paths = ['src/website1.html', 
                 'src/website2.html',
                 'src/website3.html']

# Extract column names
def extract_column_from_header(row):
    if row.br: row.br.extract()
    if row.a: row.a.extract()
    if row.sup: row.sup.extract()

    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name  

for i_web, path in enumerate(website_paths):
    with open(path, 'r') as f:
        html_data = f.read()
    soup = BeautifulSoup(html_data)
    html_table = soup.find_all('table')[0]

    # Extract column names
    if i_web == 0:
        column_names = []
        for row in html_table.find_all('th'):
            name = extract_column_from_header(row)
            if name is not None and len(name) > 0:
                column_names.append(name)
    
    product_dict = dict.fromkeys(column_names)
    # Set dict value to empty list
    for key in product_dict.keys():
        product_dict[key] = []  

    for rows in html_table.find_all('tr'):
        # skip table heading
        if rows.th: continue
        row = rows.find_all('td')
        for i, col_name in enumerate(column_names):
            product_dict[col_name].append(row[i].contents[0])
    
    if i_web == 0:
        result = pd.DataFrame(product_dict)
        result['CarPrice'] = result['CarPrice'].astype('int')
        result.rename(columns={"CarPrice": "CarPrice_{0}".format(i_web+1)}, 
                      inplace=True)
    else:
        temp = pd.DataFrame(product_dict)
        temp['CarPrice'] = temp['CarPrice'].astype('int')
        temp.rename(columns={"CarPrice": "CarPrice_{0}".format(i_web+1)}, 
                    inplace=True)
        result = pd.merge(result, temp, how='inner', on=['SKU', 'SupplierName'])

In [276]:
result[result['SKU'].str.contains('TOYOTA-4RUNNER')]

,SKU,SupplierName,CarPrice_1,CarPrice_2,CarPrice_3
132,TOYOTA-4RUNNER-MAUV-2006,Edgeify,580273,555769,560871
183,TOYOTA-4RUNNER-MAUV-2000,Realmix,560901,543835,548619
352,TOYOTA-4RUNNER-PINK-2000,Thoughtbeat,843069,839220,730419
